In [12]:
import pandas as pd
import numpy as np
import json
import os
from dotenv import load_dotenv
from llama_index.experimental.query_engine import PandasQueryEngine
from openai import OpenAI
client = OpenAI(
  organization='org-zgxvi5rnAEf6v1uGqDpdR8wR',
  project='$PROJECT_ID',
)
load_dotenv()
OPENAI_GPT4O_MODEL='gpt-4o'
OPENAI_EMBED_MODEL='text-embedding-ada-002'

In [45]:
assistant = client.beta.assistants.create(
  name="Restaurant Recommendation",
  instructions="You are an expert restaurant recommender. Please recommend restaurants to users based on information provided.",
  model="gpt-4o",
  tools=[{"type": "file_search"}],
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/assistants "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/assistants "HTTP/1.1 200 OK"


In [46]:
# Create a vector store caled "Financial Statements"
vector_store = client.beta.vector_stores.create(name="Restaurant Reviews")
 
# Ready the files for upload to OpenAI
file_paths = ["my_dataframe.txt"]
file_streams = [open(path, "rb") for path in file_paths]
 
# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)
 
# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)
print(file_batch)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/vector_stores "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/vector_stores "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/files "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/files "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/vector_stores/vs_Pfsj8rgqSpq2aQconCbX4Tax/file_batches "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/vector_stores/vs_Pfsj8rgqSpq2aQconCbX4Tax/file_batches "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.openai.com/v1/vector_stores/vs_Pfsj8rgqSpq2aQconCbX4Tax/file_batches/vsfb_15456da285b545739a551fa58101b062 "HTTP/1.1 200 OK"
HTTP Request: GET https://api.openai.com/v1/vector_stores/vs_Pfsj8rgqSpq2aQconCbX4Tax/file_batches/vsfb_15456da285b545739a551fa58101b062 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.openai.com/v1/vector_stores/vs_Pfsj8rgqSpq2aQconCbX4Tax/file_batches/vsfb_154

In [47]:
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/assistants/asst_DQra6WoXhEEobZh3S188gBHG "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/assistants/asst_DQra6WoXhEEobZh3S188gBHG "HTTP/1.1 200 OK"


In [48]:
from typing_extensions import override
from openai import AssistantEventHandler, OpenAI
 
client = OpenAI()
 
class EventHandler(AssistantEventHandler):
    @override
    def on_text_created(self, text) -> None:
        print(f"\nassistant > ", end="", flush=True)

    @override
    def on_tool_call_created(self, tool_call):
        print(f"\nassistant > {tool_call.type}\n", flush=True)

    @override
    def on_message_done(self, message) -> None:
        # print a citation to the file searched
        message_content = message.content[0].text
        annotations = message_content.annotations
        citations = []
        for index, annotation in enumerate(annotations):
            message_content.value = message_content.value.replace(
                annotation.text, f"[{index}]"
            )
            if file_citation := getattr(annotation, "file_citation", None):
                cited_file = client.files.retrieve(file_citation.file_id)
                citations.append(f"[{index}] {cited_file.filename}")

        print(message_content.value)
        print("\n".join(citations))


# Then, we use the stream SDK helper
# with the EventHandler class to create the Run
# and stream the response.

with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please address the user as Jane Doe. The user has a premium account.",
    event_handler=EventHandler(),
) as stream:
    stream.until_done()

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/threads/thread_BUyJyu4wrmybJShTogGriLvH/runs "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/threads/thread_BUyJyu4wrmybJShTogGriLvH/runs "HTTP/1.1 200 OK"

assistant > Is there anything else you would like to explore about these restaurants, Jane Doe?



In [28]:
PROMPT = """
You are a personal assistant that recommends restaurants to users.
You will take in a query and return a list of restaurants that match what the user is asking for.
Here is an example, delimited by ###:

User: Show me some italian restaurants near me with a rating above 4.5.
Output: 
Gypsy's Trattoria & Pizzeria: 4.4 Rating
Pasta Bene: 4.3 Rating


"""

completion = client.chat.completions.create(
    model=OPENAI_GPT4O_MODEL,
    prompt="You are a personal assistant that recommends restaurants to users. "
)

print(completion.choices[0].message.content)

TypeError: Missing required arguments; Expected either ('messages' and 'model') or ('messages', 'model' and 'stream') arguments to be given

In [ ]:
df = pd.read_csv("/Users/pulakdugar/Documents/pearhackathon/reviews.csv")
df = df.drop(["Time", "Pictures", "7514"], axis = 1)

In [ ]:
df.head()

,Restaurant,Reviewer,Review,Rating,Metadata
0,Beyond Flavours,Rusha Chakraborty,"The ambience was good, food was quite good . h...",5,"1 Review , 2 Followers"
1,Beyond Flavours,Anusha Tirumalaneedi,Ambience is too good for a pleasant evening. S...,5,"3 Reviews , 2 Followers"
2,Beyond Flavours,Ashok Shekhawat,A must try.. great food great ambience. Thnx f...,5,"2 Reviews , 3 Followers"
3,Beyond Flavours,Swapnil Sarkar,Soumen das and Arun was a great guy. Only beca...,5,"1 Review , 1 Follower"
4,Beyond Flavours,Dileep,Food is good.we ordered Kodi drumsticks and ba...,5,"3 Reviews , 2 Followers"


In [ ]:
def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding


df['ada_embedding'] = df.combined.apply(lambda x: get_embedding(x, model='text-embedding-3-small'))
df.to_csv('output/embedded_1k_reviews.csv', index=False)

AttributeError: 'DataFrame' object has no attribute 'combined'

In [ ]:
data = {
    'Restaurant': ['Chez Panisse', 'La Note', 'Ippuku', 'Comal', 'Gather'],
    'Location': ['1517 Shattuck Ave', '2377 Shattuck Ave', '2130 Center St', '2020 Shattuck Ave', '2200 Oxford St'],
    'Rating': [4.1, 4.4, 4.5, 4.4, 4.3],  # Ratings out of 5 stars
    'Review Count': [3180, 2100, 1400, 1130, 3560],
    'Cuisine': ['Californian/French', 'French', 'Japanese (Izakaya)', 'Mexican', 'American/Farm-to-Table'],
    'Price Range': ['$$$$', '$$', '$$$', '$$', '$$'],
    'Operating Hours': [
        'Mon-Sat: 11:30am - 2:30PM, 5 PM - 10 PM, Sun: Closed',
        'Daily: 8 AM - 2 PM',
        'Tue-Sun: 5 PM - 10 PM, Mon: Closed',
        'Daily: 11 AM - 10 PM',
        'Wed-Fri: 11 AM - 2 PM, 4 PM - 8:30 PM, Sat-Sun: 10 AM - 2 PM, 4 PM - 8:30 PM, Mon-Tue: Closed'
    ],
    'Contact': [
        '(510) 548-5525',
        '(510) 848-0240',
        '(510) 843-3533',
        '(510) 644-0400',
        '(510) 809-0400'
    ]
}

# Create a pandas DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df)

     Restaurant           Location  Rating  Review Count  \
0  Chez Panisse  1517 Shattuck Ave     4.1          3180   
1       La Note  2377 Shattuck Ave     4.4          2100   
2        Ippuku     2130 Center St     4.5          1400   
3         Comal  2020 Shattuck Ave     4.4          1130   
4        Gather     2200 Oxford St     4.3          3560   

                  Cuisine Price Range  \
0      Californian/French        $$$$   
1                  French          $$   
2      Japanese (Izakaya)         $$$   
3                 Mexican          $$   
4  American/Farm-to-Table          $$   

                                     Operating Hours         Contact  
0  Mon-Sat: 11:30am - 2:30PM, 5 PM - 10 PM, Sun: ...  (510) 548-5525  
1                                 Daily: 8 AM - 2 PM  (510) 848-0240  
2                 Tue-Sun: 5 PM - 10 PM, Mon: Closed  (510) 843-3533  
3                               Daily: 11 AM - 10 PM  (510) 644-0400  
4  Wed-Fri: 11 AM - 2 PM, 4 PM - 8:30